In [107]:
!pip install spotipy
!pip install XlsxWriter
import os
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
os.environ['SPOTIPY_CLIENT_ID']='c98e011c0f6a4d5b8ed0cf4caef922c4'
os.environ['SPOTIPY_CLIENT_SECRET']='058ecc9c70a04bb4acdf19673e96af29'
spotify=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [108]:
df=pd.read_csv('drive/My Drive/MoodPlaylists.csv')

In [109]:
mainsongsdf=pd.DataFrame(columns={'emotion','track','album','artists','popularity'})

In [110]:
for i in range(len(df)):
    emo=df.loc[i]['emotion']
    id=df.loc[i]['playlistid']
    results=spotify.playlist(id,additional_types=('track',))
    playdict={'emotion':emo,'track':[],'album':[],'artists':[],'popularity':[]}
    for i in results['tracks']['items']:
        playdict['track'].append(i['track']['name'])
        playdict['album'].append(i['track']['album']['name'])
        s=''
        for j in i['track']['artists']:
            s=s+j['name']+', '
        playdict['artists'].append(s[:len(s)-2])
        playdict['popularity'].append(i['track']['popularity'])
    playdf=pd.DataFrame(playdict)
    playdf=playdf.sort_values(by=['popularity'],ascending=False).reset_index(drop=True)
    mainsongsdf=mainsongsdf.append(playdf)
mainsongsdf=mainsongsdf.reset_index(drop=True)

In [111]:
print(mainsongsdf)

     popularity  ...                                              album
0            79  ...                                            Fanmail
1            72  ...                               B'Day Deluxe Edition
2            69  ...                                      New Beginning
3            64  ...                                        Bittersweet
4            63  ...                             Pearl (Legacy Edition)
...         ...  ...                                                ...
1227          0  ...                                           Geronimo
1228          0  ...                                         All Of You
1229          0  ...                              The Midsummer Station
1230          0  ...                                    Oh, What A Life
1231          0  ...  The Beginning & The Best Of The E.N.D. (Intern...

[1232 rows x 5 columns]


In [113]:
writer=pd.ExcelWriter('drive/My Drive/EmotionSongs.xlsx',engine='xlsxwriter')
mainsongsdf.to_excel(writer,sheet_name='Sheet1',index=False)
writer.save()